# Исследовательский проект по НИС «Анализ данных в Python»
<div style="text-align: right">
    Проект подготовлен студентами БПИ228
    <br>
    Лысиным Кириллом и Гакал Анжеликой
</div>

## Часть 2. Сбор данных из интернета и составление датасета.
Данную часть задания мы выполняли первой, так как её результаты являются предметом аналаза в следующей части задания.

После обсуждения вариантов мы решили собирать информацию о товарах и услугах, размещённых на сервисе объявлений Юла.
Выбором данного сервиса объявлений обусловлен ряд решений в области реализации:
<br>
<p style="margin-left: 50px;">
    <ul>
        <li>Библиотека requests, пройденная на занятиях, не позволяет получить все необходимые данные со страницы, так как часть контента отображается только после выполнения js-скриптов, что не происходит при использовании вышеуказанной библиотеки, поэтому для получения данных с сайта исопльзуется библиотека selenium.</li>
        <li>Работа с динамически генерируемым содержимым сайта делает невозможной (или крайне затруднительной) унификацию процесса, так как для разных устройств это содержимое может отличаться. Этим фактом обусловлена заточенность прогаммы, осуществляющей парсинг, под конкретное устройство, но изменения, необходимые для работы программы на другом устройстве могут быть внесены за незначительный промежуток времени.</li>
    </ul>
</p>

In [2]:
import datetime
from selenium import webdriver
import pandas as pd
import time
from bs4 import BeautifulSoup
import os.path


    
driver = webdriver.Chrome()

with open('visited_links.txt', 'w+') as f:
    visited_links = set(f.readlines())


def get_item_info(url):
    
    info_dict = {}
    soup = get_soup(url)
    if(soup == None):
        return
    
    general_info = str(soup.findAll('meta', {'property':"og:description"})[0])

    # Сохранение данных о товаре/услуге в словарь
    info_dict['Name'] = get_item_name(general_info)
    info_dict['Price (in rubles)'] = get_item_price(general_info)
    info_dict['Category'] = get_item_category(general_info)
    info_dict['Region'] = get_item_region(soup)
    info_dict['Time/date of placement'] = get_item_placement_time(soup)
    info_dict['Added to favourites'] = get_favourites(soup)
    info_dict['Number of seller\'s ads'] = get_number_of_ads(soup)
    info_dict['Seller rating'] = get_seller_rating(soup)
    info_dict['Number of views'] = get_number_of_views(soup)
    info_dict['Number of images'] = get_number_of_images(soup)
    info_dict['Link'] = url
    
    return info_dict

def get_soup(url, counter = 0):

    time.sleep(1)
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        return soup
    # Если не удалось сразу получить страницу, то программа пробует еще несколько раз
    except:
        if(counter < 3):
            return get_soup(url, counter + 1)
    # Если после несольких попыток так и не удалось считать содержимое страницы, то этот метод вернет None

def get_item_name(info):
    try:
        name = info[15:info.find('–') - 1]
        return name
    except:
        return None

def get_item_price(info):
    try:
        if not('Цена: ' in info):
            return 0
        price = float(info[info.find('Цена: ') + 6:info.find('руб.') - 1].replace(' ', ''))
        return price
    except:
        return 0

def get_item_category(info):
    try:
        category = info[info.find('раздела «') + 9:info.find('».')]
        return category
    except:
        return None

def get_item_region(soup):
    try:
        region = list(soup.findAll('span', class_='sc-cOxWqc hNYaaO'))[0].text
        return region
    except:
        return None

def get_item_placement_time(soup):
    try:
        s = list(soup.findAll('dd', class_='sc-cOxWqc sc-fVmuvm eOPaPs dVayGV'))[-1].text
        now = datetime.datetime.now()
        if('Сегодня в' in s):
            return datetime.datetime(now.year, now.month, now.day, int(s[-5:-3]), int(s[-2:]))
        if('Вчера в' in s):
            return (datetime.datetime(now.year, now.month, now.day, int(s[-5:-3]), int(s[-2:])) + datetime.timedelta(days=-1))
        if('Позавчера в' in s):
            return (datetime.datetime(now.year, now.month, now.day, int(s[-5:-3]), int(s[-2:])) + datetime.timedelta(days=-2))


        months = ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
        for j in range(len(months)):
            if(months[j].lower() in s):
                return datetime.datetime(now.year, j + 1, int(s[:2]), 12)
        return datetime.datetime(day=int(s[:2]), month=int(s[3:5]), year=int(s[6:10]), hour = 12)
    except:
        return None


def get_favourites(soup):
    try:
        favourites_count = list(soup.findAll('dd', class_="sc-cOxWqc sc-fVmuvm eOPaPs dVayGV"))
        return int(favourites_count[-3].text)
    except:
        return None

def get_number_of_ads(soup):
    try:
        n = list(soup.findAll('span', {'data-test-component':"UserNameClick"}))[0].text
        n = int(n[n.find('(')+1:n.find(' объявл')])
        return n
    except:
        return None

def get_seller_rating(soup):
    try:
        rating = soup.find('span', class_="sc-cOxWqc eDoIYl")
        return float(rating.text.replace(',', '.')) if rating != None else None
    except:
        return None

def get_number_of_views(soup):
    try:
        number_of_views = list(soup.findAll('dd', class_='sc-cOxWqc sc-fVmuvm eOPaPs dVayGV'))
        return int(number_of_views[-2].text)
    except:
        return None

def get_number_of_images(soup):
    try:
        images = list(soup.findAll('img', class_="sc-fBnnfK"))
        return len(images)
    except:
        return None

def get_links(scroll_count):
    
    url = r'https://youla.ru/'
    driver.get(url)
    time.sleep(1)
    
    for i in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    links = [str(i.a) for i in soup.findAll('span', class_="sc-llGDqb sc-gqgnwQ fEAASo hZGRky")]
    links = [r'https://youla.ru' + s[s.find(r'href="') + 6 : s.find(r'" rel')] for s in links]
    return links



n = 50
for i in range(n):
    items = []
    print(f"Processing batch {i + 1} out of {n}:", end='')
    links = get_links(1)
    for i in range(len(links)):
        if(int(i * 100/len(links))//5 > int((i - 1) * 100/len(links))//5):
            print(' .', end='')
        link = links[i]
        if(link in visited_links):
            continue
        items.append(get_item_info(link))
        visited_links.add(link)
    print(' ✓')

    df = pd.DataFrame(items)
    if(len(df) > 0):
        df = df.set_index('Name')
        if(os.path.isfile('items.xlsx')):
            prev_df = pd.read_excel('items.xlsx', index_col = 0)
            df = pd.concat([prev_df, df])

        df.to_excel('items.xlsx')

print("Done! The data has been saved to the file items.xlsx")

with open('visited_links.txt', 'a') as f:
    for line in visited_links:
        f.write(f"{line}\n")



driver.quit()
df

Processing batch 1 out of 50: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 2 out of 50: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 3 out of 50: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 4 out of 50: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 5 out of 50: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 6 out of 50: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 7 out of 50: . . . . . . . . . . . . ✓
Processing batch 8 out of 50: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 9 out of 50: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 10 out of 50: . . . . . . . . . . . . ✓
Processing batch 11 out of 50: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 12 out of 50: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 13 out of 50: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 14 out of 50: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 15

,Price (in rubles),Category,Region,Time/date of placement,Added to favourites,Number of seller's ads,Seller rating,Number of views,Number of images,Link
Name,,,,,,,,,,
Брюки 80-92,0.0,Детский гардероб,Москва,2023-01-31 22:00:00,2.0,40.0,5.0,14.0,5,https://youla.ru/moskva/detskaya-odezhda/shtan...
Туфли размер 30. Бесплатно,0.0,Детский гардероб,Химки,2023-01-31 21:59:00,0.0,54.0,5.0,18.0,0,https://youla.ru/himki/detskaya-odezhda/obuv/t...
Кошка бесплатно,0.0,Животные,Москва,2023-01-31 21:59:00,7.0,1.0,NaN,307.0,3,https://youla.ru/all/zhivotnye/koshki/bielaia-...
Demix на 6 лет на синтепоне,0.0,Детский гардероб,Котельники,2023-01-31 21:59:00,0.0,111.0,5.0,127.0,4,https://youla.ru/kotelniki/detskaya-odezhda/ve...
Красивый пёс в добрые руки,0.0,Животные,Люберцы,2023-01-31 21:59:00,151.0,19.0,NaN,8402.0,10,https://youla.ru/lyubertsy/zhivotnye/sobaki/kr...
...,...,...,...,...,...,...,...,...,...,...
Очки без диоптрий,300.0,Красота и здоровье,Москва,2023-02-03 17:17:00,0.0,3.0,5.0,0.0,4,https://youla.ru/moskva/krasota-i-zdorove/medi...
Новое платье паетки,1800.0,Женский гардероб,Москва,2023-02-03 11:01:00,0.0,173.0,5.0,1.0,5,https://youla.ru/moskva/zhenskaya-odezhda/plat...
Люстра 80-е годы,3000.0,Для дома и дачи,Москва,2023-01-27 12:00:00,0.0,6.0,5.0,15.0,3,https://youla.ru/moskva/dom-dacha/osveshchenie...


## Часть 1. Анализ датасета.

Загружаем содержимое файла items.xlsx, в котором содержатся данные, полученные в ходе выполнения части 2 проекта.  
Переименуем некоторые столбцы для удобства и выведем первые 5 строк датафрейма.

In [35]:
import pandas as pd

df = pd.read_excel("items.xlsx")
# Переименуем названия столбцов датасета для более удобного доступа к ним
df = df.rename(
    columns={
        'Time/date of placement': 'Placement_time',
        'Added to favourites': 'Favourites_count',
        'Number of views':'Views',
        'Number of images':'Images',
        'Price (in rubles)':'Price',
        'Number of seller\'s ads':'Ads_count',
        'Seller rating':'Seller_rating'
    })
df.head()


,Name,Price,Category,Region,Placement_time,Favourites_count,Ads_count,Seller_rating,Views,Images,Link
0,Брюки 80-92,0.0,Детский гардероб,Москва,2023-01-31 22:00:00,2.0,40.0,5.0,14.0,5,https://youla.ru/moskva/detskaya-odezhda/shtan...
1,Туфли размер 30. Бесплатно,0.0,Детский гардероб,Химки,2023-01-31 21:59:00,0.0,54.0,5.0,18.0,0,https://youla.ru/himki/detskaya-odezhda/obuv/t...
2,Кошка бесплатно,0.0,Животные,Москва,2023-01-31 21:59:00,7.0,1.0,NaN,307.0,3,https://youla.ru/all/zhivotnye/koshki/bielaia-...
3,Demix на 6 лет на синтепоне,0.0,Детский гардероб,Котельники,2023-01-31 21:59:00,0.0,111.0,5.0,127.0,4,https://youla.ru/kotelniki/detskaya-odezhda/ve...
4,Красивый пёс в добрые руки,0.0,Животные,Люберцы,2023-01-31 21:59:00,151.0,19.0,NaN,8402.0,10,https://youla.ru/lyubertsy/zhivotnye/sobaki/kr...


### Описание датасета

<i><b>- Определим размер датасета:</b></i>

In [36]:
size = df.shape
print(f"Количество строк (наблюдений): {size[0]}\n"+
     f'Количество столбцов (переменных): {size[1]}')

Количество строк (наблюдений): 10931
Количество столбцов (переменных): 11


<b><i>- Опишем переменные:</b></i>

In [37]:
columns = df.columns.to_list()
column_types = [str(i).ljust(15, ' ') for i in df.dtypes.to_list()]
na_count = [str(i) for i in df.isna().sum()]

s = [str(i) for i in df['Category']]
df['Category'] = s;
print("В датасете содержатся следующие переменные:")
print(*[('\t' + i).ljust(20, ' ') + "|тип данных переменной: " + j + "|число пустых значений: " + k for i, j, k in zip(columns, column_types, na_count)], sep='\n')

В датасете содержатся следующие переменные:
	Name               |тип данных переменной: object         |число пустых значений: 0
	Price              |тип данных переменной: float64        |число пустых значений: 0
	Category           |тип данных переменной: object         |число пустых значений: 1
	Region             |тип данных переменной: object         |число пустых значений: 35
	Placement_time     |тип данных переменной: datetime64[ns] |число пустых значений: 311
	Favourites_count   |тип данных переменной: float64        |число пустых значений: 308
	Ads_count          |тип данных переменной: float64        |число пустых значений: 377
	Seller_rating      |тип данных переменной: float64        |число пустых значений: 4916
	Views              |тип данных переменной: float64        |число пустых значений: 35
	Images             |тип данных переменной: int64          |число пустых значений: 0
	Link               |тип данных переменной: object         |число пустых значений: 0


<p style="margin-left:5em;">
    <ul>
        <li><em>Price</em> - цена товара/услуги в рублях. Метрическая переменная</li>  
        <li><em>Category</em> - категория товара. Категориальная переменная (Номинальная)</li>
        <li><em>Region</em> - местоположение продавца. Категориальная переменная (Номинальная)</li>
        <li><em>Placement_time</em> - дата и время размещения объявления. Метрическая переменная</li> 
        <li><em>Favourites_count</em> - количество людей, добавивших объявление в избранное. Метрическая переменная</li>
        <li><em>Ads_count</em> - количество объявлений, которые разместил продваец на сервисе. Метрическая переменная</li>
        <li><em>Seller_rating</em> - рейтинг продавца по пятибалльной шкале. Категориальная переменная (Порядковая)</li>
        <li><em>Views</em> - количество просмотров объявления. Метрическая переменная</li>
        <li><em>Images</em> - количество фотографий, прикреплённых к объявлению. Метрическая переменная</li>
        <li><em>Link</em> - ссылка на объявление</li>
    </ul>
</p>

<b><i>- Изменение типов данных:</i></b>
<br>Для данного датасета не нужно изменять типы данных в столбцах, так как он был составлен с учётом необходимости дальнейшего анализа.

### Добавление новых столбцов

Добавим новые колонки:
<ul>
    <li><em>Favourites_percent</em>, отражающую процент тех пользователей, которые добавили товар в избранное из числа людей, просмотревших объявление. (Категориальная (порядковая) переменная, тип значений в столбце - float64)</li>
    <li><em>Free_item</em>, показывающую, является ли товар/услуга бесплатной. (Категориальная (бинарная) переменная, тип значений в столбце - bool)</li>
</ul>

In [38]:
favourites_percent = [i/j * 100 if j != 0 else None for i, j in zip(df["Favourites_count"], df["Views"])]
free = [i == 0 for i in df["Price"]]
df['Favourites_percent'] = favourites_percent
df['Free_item'] = free
df.head()

,Name,Price,Category,Region,Placement_time,Favourites_count,Ads_count,Seller_rating,Views,Images,Link,Favourites_percent,Free_item
0,Брюки 80-92,0.0,Детский гардероб,Москва,2023-01-31 22:00:00,2.0,40.0,5.0,14.0,5,https://youla.ru/moskva/detskaya-odezhda/shtan...,14.285714,True
1,Туфли размер 30. Бесплатно,0.0,Детский гардероб,Химки,2023-01-31 21:59:00,0.0,54.0,5.0,18.0,0,https://youla.ru/himki/detskaya-odezhda/obuv/t...,0.000000,True
2,Кошка бесплатно,0.0,Животные,Москва,2023-01-31 21:59:00,7.0,1.0,NaN,307.0,3,https://youla.ru/all/zhivotnye/koshki/bielaia-...,2.280130,True
3,Demix на 6 лет на синтепоне,0.0,Детский гардероб,Котельники,2023-01-31 21:59:00,0.0,111.0,5.0,127.0,4,https://youla.ru/kotelniki/detskaya-odezhda/ve...,0.000000,True
4,Красивый пёс в добрые руки,0.0,Животные,Люберцы,2023-01-31 21:59:00,151.0,19.0,NaN,8402.0,10,https://youla.ru/lyubertsy/zhivotnye/sobaki/kr...,1.797191,True


<b><em>- Перекодирование переменных</em></b>
<br>
Добавим столбец <em>Price_range</em>, в котором перекодируем метрическую переменную <em>Price</em> в категориальную.
Все цены разделели на 5 категорий:
<ul>
    <li><em>free</em> - бесплатные товары</li>
    <li><em>small price</em> - недорогие товары (стоимостью до 5000 рублей)</li>
    <li><em>affordable</em> - товары среднего ценового сегмента (от 5000 до 15000 рублей)</li>
    <li><em>expensive</em> - дорогие товары (от 15000 до 100000 рублей)</li>
    <li><em>luxury</em> - товары роскоши (стоимостью более 100000 рублей)</li>
</ul>

In [39]:
price_range = ["free" if x == 0 else "small price" if x <= 5000 else "affordable" if x <= 15000 else "expensive" if x <= 100000 else "luxury" for x in df['Price']]
df.insert(2, 'Price_range', price_range)

In [41]:
df.head(10)

,Name,Price,Price_range,Category,Region,Placement_time,Favourites_count,Ads_count,Seller_rating,Views,Images,Link,Favourites_percent,Free_item
0,Брюки 80-92,0.0,free,Детский гардероб,Москва,2023-01-31 22:00:00,2.0,40.0,5.0,14.0,5,https://youla.ru/moskva/detskaya-odezhda/shtan...,14.285714,True
1,Туфли размер 30. Бесплатно,0.0,free,Детский гардероб,Химки,2023-01-31 21:59:00,0.0,54.0,5.0,18.0,0,https://youla.ru/himki/detskaya-odezhda/obuv/t...,0.000000,True
2,Кошка бесплатно,0.0,free,Животные,Москва,2023-01-31 21:59:00,7.0,1.0,NaN,307.0,3,https://youla.ru/all/zhivotnye/koshki/bielaia-...,2.280130,True
3,Demix на 6 лет на синтепоне,0.0,free,Детский гардероб,Котельники,2023-01-31 21:59:00,0.0,111.0,5.0,127.0,4,https://youla.ru/kotelniki/detskaya-odezhda/ve...,0.000000,True
4,Красивый пёс в добрые руки,0.0,free,Животные,Люберцы,2023-01-31 21:59:00,151.0,19.0,NaN,8402.0,10,https://youla.ru/lyubertsy/zhivotnye/sobaki/kr...,1.797191,True
5,!Зимний конверт на выписку.,0.0,free,Детский гардероб,Апрелевка,2023-01-31 21:59:00,7.0,4.0,5.0,499.0,3,https://youla.ru/aprelevka/detskaya-odezhda/ko...,1.402806,True
6,Диски для гантелей,150.0,small price,Спорт и отдых,Москва,2023-01-31 14:51:00,0.0,607.0,5.0,3.0,3,https://youla.ru/moskva/sport-otdyh/trenazhery...,0.000000,False
7,6 16 коробка,100000.0,expensive,Легковые автомобили,Москва,2023-01-31 17:22:00,1.0,1.0,NaN,13.0,6,https://youla.ru/moskva/auto/s-probegom/6-16-k...,7.692308,False
8,"Подушки Бамбук новые, в упаковках",450.0,small price,Для дома и дачи,Москва,2023-01-31 12:58:00,160.0,9.0,NaN,3060.0,0,https://youla.ru/moskva/dom-dacha/tekstil-kovr...,5.228758,False
9,Щенки той пуделя Ркф -FCI,0.0,free,Животные,Москва,2023-01-28 12:00:00,11.0,1.0,NaN,476.0,5,https://youla.ru/moskva/zhivotnye/sobaki/shchi...,2.310924,True


<b><em>- Формирование среза таблицы и его транспонирование</em></b>

Выберем случайные 15 подряд идущих строк из датафрейма и транспонируем их.

In [45]:
import random as rand
k = rand.randint(0, size[0] - 15)
df.loc[k:k+15, :].T

,4531,4532,4533,4534,4535,4536,4537,4538,4539,4540,4541,4542,4543,4544,4545,4546
Name,Коврик для фитнеса и тренажеров Torneo A-913G,"Ботильоны, полусапоги",Прицел оптический,xiaomi 11t pro,Ковер ручной работы пчелка,"Найк теннисная одежда, футболка, топ",Вязальная машина Brother KH260 KR260,Мальчик Хайленд,Кружевное полотно+ кружево+ ажурные воротнички,Samsung Galaxy A73,Босоножки Карло Пазолини 37,Ваз - 2115,Пуховик massimo dutti,Мандариновое дерево Кумкват 80 см,Щенки Сиба ину,"Кепка, шляпа женские велюр, фетр"
Price,3000.0,2000.0,2600.0,17000.0,6990.0,500.0,42000.0,12000.0,1750.0,5900.0,1000.0,0.0,5000.0,9990.0,15000.0,1500.0
Price_range,small price,small price,small price,expensive,affordable,small price,expensive,affordable,small price,affordable,small price,free,small price,affordable,affordable,small price
Category,Спорт и отдых,Женский гардероб,Спорт и отдых,Телефоны и планшеты,Детские товары,Женский гардероб,Бытовая техника,Животные,Хобби и развлечения,Телефоны и планшеты,Женский гардероб,Легковые автомобили,Женский гардероб,Для дома и дачи,Животные,Женский гардероб
Region,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Москва,Балашиха,Москва,Москва,Москва,Москва,Москва,Москва,Москва
Placement_time,2023-02-01 11:24:00,2023-01-23 12:00:00,2023-01-18 12:00:00,2023-01-28 12:00:00,2023-02-02 12:20:00,2023-01-20 12:00:00,2023-01-09 12:00:00,2023-01-31 15:40:00,2023-01-30 12:00:00,2023-01-24 12:00:00,2023-01-29 12:00:00,2023-01-24 12:00:00,2023-02-01 21:56:00,2023-02-02 15:50:00,2023-01-06 12:00:00,2023-01-07 12:00:00
Favourites_count,0.0,3.0,10.0,7.0,0.0,2.0,3.0,0.0,15.0,8.0,1.0,2.0,0.0,0.0,25.0,2.0
Ads_count,17.0,4.0,3.0,1.0,NaN,11.0,6.0,38.0,3.0,350.0,25.0,2.0,6.0,2.0,2.0,75.0
Seller_rating,5.0,4.4,NaN,NaN,NaN,5.0,NaN,5.0,5.0,4.7,5.0,NaN,5.0,5.0,5.0,5.0
Views,1.0,57.0,144.0,69.0,8.0,52.0,100.0,29.0,520.0,172.0,22.0,120.0,22.0,19.0,271.0,57.0


<b><em> - Многоуровневая сортировка</em></b>

Отсортируем датафрейм сначала по убыванию цены, а затем по убыванию количества просмотров. После этого выведем первые 20 наблюдений из отсортированного датасета.

In [55]:
sortedDf = df.dropna(subset = ['Price', 'Views']).sort_values(['Price', 'Views'], ascending=[False, False])
sortedDf.iloc[:20, :][['Price', 'Views']]

,Price,Views
7412,74900000.0,65.0
6654,40000000.0,2228.0
2321,40000000.0,2211.0
570,40000000.0,2184.0
9324,15000000.0,6041.0
5739,15000000.0,6038.0
5259,15000000.0,6036.0
7377,12000000.0,652.0
3547,12000000.0,648.0
2895,12000000.0,632.0


## Цель и задачи исследования

<b>Цель исследования:</b> проанализровать рынок товаров одного из самых крупных маркетплейсов в России - Юла.
<br><br>
<b>Задачи исследования:</b>
<ol>
    <li>Сформировать исследовательские гипотезы</li>
    <li>Построить графики, отражающие зависимость между выбранными переменными датасета</li>
    <li>Проанализировать данные, полученные на предыдущих этапах исследования</li>
    <li>Подтвердить или опровергуть поставленные гипотезы</li>
</ol>

<b><em>- Формирование исследовательских гипотез</em></b>